>⚠️ This notebook is using the development version of `visual_blocks` from the [main branch](https://github.com/wrighkv1/visual-blocks/tree/main) in Github as opposed to a release version from PyPI.

# Quick-Start Notebook  [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/wrighkv1/visual-blocks/blob/main/tests/main.ipynb)

Visual Blocks is a graphical development environment running in a Jupyter notebook cell. Create inteactive ML demos by mixing and matching built-in and custom blocks. Custom blocks call code found in other cells of the notebook.

Visual Blocks is distributed as a Python package. To begin, install it with `!pip`.

In [ ]:
!pip -q install "git+https://github.com/wrighkv1/visual-blocks.git@main#subdirectory=python"

##### Next write an inference function to visualize with Visual Blocks. Here we use the Tensorflow Lite interpreter with a CartoonGAN model to transform an image.

In [ ]:
import numpy as np
import tensorflow as tf
import urllib.request

MODEL_URL = "https://storage.googleapis.com/tfweb/models/cartoongan_fp16.tflite"

with urllib.request.urlopen(MODEL_URL) as model:
  interpreter = tf.lite.Interpreter(model_content=model.read())

def cartoongan(tensors):
  """Inference function for use with Visual Blocks.

  This function is passed to the Visual Blocks server, which calls it to
  implement a Colab model runner block.

  Args:
    tensors: An input image tensor in Visual Blocks's list-of-dicts format. The
    image must be a single floating point tensor of size 224x224x3 with pixel
    values from -1 to 1.

  Returns: 
    The output image tensor Visual Blocks's list-of-dicts format. The image is
    returned in a single uint8 tensor of size 1x224x224x3 with pixel values
    from 0 to 255.
  """

  tensor_values = tensors[0]['tensorValues']
  tensor_shape = tensors[0]['tensorShape']
  tensor = tf.constant(tensor_values, shape=tensor_shape)
  if len(tensor_shape) == 3:
    tensor = tf.expand_dims(tensor, axis=0)

  interpreter.allocate_tensors()
  input_details = interpreter.get_input_details()
  interpreter.set_tensor(input_details[0]['index'], tensor)

  interpreter.invoke()

  raw_prediction = interpreter.tensor(
      interpreter.get_output_details()[0]['index'])()

  output = (np.squeeze(raw_prediction)+1.0)*127.5
  output = np.clip(output, 0, 255).astype(np.uint8)

  return {
    'tensors': [
      {
        'tensorValues': output.flatten().tolist(),
        'tensorShape': [1, 224, 224, 3],
      },
    ],
  }

##### Then start a Visual Blocks server, passing it the inference function to use in a custom block.

In [ ]:
import visual_blocks
server = visual_blocks.Server(cartoongan)
server.show_ui()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>